In [616]:
import pandas as pd
import psycopg2 as pg

In [617]:
import sqlalchemy


In [618]:
conexao = pg.connect(host='localhost', 
                 dbname='ViajenoBolso', 
                 user='postgres',
                 password='2003')

In [619]:
cursor = conexao.cursor()

In [620]:
import tkinter as tk
from tkinter import messagebox

In [621]:
import sys
print(sys.executable)


c:\ProgramData\anaconda3\python.exe


In [622]:
import tkinter as tk
from tkinter import messagebox

telefones = []


def adicionar_telefone():
    telefone = entry_telefone.get()
    if telefone:
        telefones.append(telefone)
        listbox_telefones.insert(tk.END, telefone)
        entry_telefone.delete(0, tk.END)

def cadastrar_usuario():
    primeiro_nome = entry_primeiro_nome.get()
    sobrenome = entry_sobrenome.get()
    email = entry_email.get()
    senha = entry_senha.get()
    data_nascimento = entry_data_nascimento.get()
    pais = entry_pais.get()
    estado = entry_estado.get()
    cidade = entry_cidade.get()

    if primeiro_nome == '' or email == '' or senha == '' or not telefones:
        messagebox.showwarning("Erro", "Campos obrigatórios: Nome, Email, Senha e pelo menos 1 telefone")
        return

    try:
        cursor = conexao.cursor()

        cursor.execute('''
            INSERT INTO Usuario (primeiro_nome, sobrenome, email, senha, data_nascimento, pais, estado, cidade) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s) RETURNING id
        ''', (primeiro_nome, sobrenome, email, senha, data_nascimento, pais, estado, cidade))
        usuario_id = cursor.fetchone()[0] 

        for telefone in telefones:
            cursor.execute('''
                INSERT INTO Telefone (usuario_id, telefone)
                VALUES (%s, %s)
            ''', (usuario_id, telefone))

        conexao.commit()

        dados_cadastro = f"""
        Cadastro realizado com sucesso!
        
        Nome: {primeiro_nome} {sobrenome}
        Email: {email}
        Telefone: {', '.join(telefones)}
        Senha: {senha}
        Data de Nascimento: {data_nascimento}
        País: {pais}
        Estado: {estado}
        cidade: {cidade}
        """
        messagebox.showinfo("Dados do Cadastro", dados_cadastro)

        limpar_campos()
    except psycopg2.IntegrityError:
        messagebox.showerror("Erro", "Erro ao cadastrar usuário")
        conexao.rollback()

def abrir_tela_configuracoes():
    tela_configuracoes = tk.Toplevel(app)
    tela_configuracoes.title("Configurações")

    tk.Label(tela_configuracoes, text="Buscar por Nome:").grid(row=1, column=0, padx=10, pady=5)
    entry_busca_nome = tk.Entry(tela_configuracoes)
    entry_busca_nome.grid(row=1, column=1, padx=10, pady=5)
    tk.Button(tela_configuracoes, text="Buscar", command=lambda: buscar_usuario_por_nome(entry_busca_nome.get())).grid(row=1, column=2, padx=10, pady=5)
    tk.Button(tela_configuracoes, text="Retornar", bg="blue", fg="white", command=tela_configuracoes.destroy).grid(row=8, columnspan=3, pady=10)
    tk.Button(tela_configuracoes, text="Atualizar Usuário", command=lambda: atualizar_usuario(usuario_id_selecionado)).grid(row=6, columnspan=3, pady=10)


   

def buscar_usuario_por_nome(nome):
    try:
        cursor = conexao.cursor()
        cursor.execute('SELECT * FROM Usuario WHERE primeiro_nome ILIKE %s', (f'%{nome}%',))
        usuarios = cursor.fetchall()

        if usuarios:
            tela_resultados = tk.Toplevel(app)
            tela_resultados.title("Resultados da Busca")

            tree = ttk.Treeview(tela_resultados, columns=("ID", "Nome", "Sobrenome", "Email", "Telefone", "Senha", "Data Nascimento", "País", "Estado", "Cidade"), show='headings')

            # Configurações das colunas
            for col in tree["columns"]:
                tree.heading(col, text=col)

            tree.column("ID", width=50)
            tree.column("Nome", width=100)
            tree.column("Sobrenome", width=100)
            tree.column("Email", width=150)
            tree.column("Telefone", width=100)
            tree.column("Senha", width=80)
            tree.column("Data Nascimento", width=80)
            tree.column("País", width=80)
            tree.column("Estado", width=150)
            tree.column("Cidade", width=150)


            for usuario in usuarios:
                cursor.execute('SELECT telefone FROM Telefone WHERE usuario_id = %s', (usuario[0],))
                telefones = cursor.fetchall()
                telefones_str = ', '.join([telefone[0] for telefone in telefones])

                # Inserindo os dados na tabela
                tree.insert('', tk.END, values=(
                    usuario[0],  
                    usuario[1],  
                    usuario[2], 
                    usuario[3],  
                    usuario[4],  
                    usuario[5], 
                    usuario[6],  
                    usuario[7],  
                    telefones_str  
                ))

            tree.pack(expand=True, fill='both', padx=5, pady=5)
        else:
            messagebox.showinfo("Resultado", "Nenhum usuário encontrado com esse nome.")
    except psycopg2.Error as e:
        messagebox.showerror("Erro", f"Erro ao buscar usuário: {e}")


def exibir_tabela_resultados(usuarios):
    tela_resultados = tk.Toplevel(app)
    tela_resultados.title("Resultados da Busca")

    tree = ttk.Treeview(tela_resultados, columns=("ID", "Nome", "Sobrenome", "Email", "Data de Nascimento", "País", "Estado", "Cidade"), show='headings')
    tree.heading("ID", text="ID")
    tree.heading("Nome", text="Primeiro Nome")
    tree.heading("Sobrenome", text="Sobrenome")
    tree.heading("Email", text="Email")
    tree.heading("Telefone", text="Telefone")
    tree.heading("Senha", text="Senha")
    tree.heading("Data Nascimento", text="Data Nascimento")
    tree.heading("País", text="País")
    tree.heading("Estado", text="Estado")
    tree.heading("Cidade", text="cidade")

    for usuario in usuarios:
        tree.insert('', tk.END, values=usuario)

    tree.pack(expand=True, fill='both', padx=10, pady=5)

    tk.Button(tela_resultados, text="Selecionar Usuário", command=lambda: selecionar_usuario(tree, tela_resultados)).pack(pady=10)

    tela_resultados.grid_rowconfigure(0, weight=1)
    tela_resultados.grid_columnconfigure(0, weight=1)


def selecionar_usuario(tree, tela_resultados):
    selected_item = tree.selection()

    if not selected_item:
        messagebox.showwarning("Aviso", "Por favor, selecione um usuário.")
        return

    usuario_selecionado = tree.item(selected_item, 'values')

    entry_primeiro_nome.delete(0, tk.END)
    entry_primeiro_nome.insert(0, usuario_selecionado[1])
    entry_sobrenome.delete(0, tk.END)
    entry_sobrenome.insert(0, usuario_selecionado[2])
    entry_email.delete(0, tk.END)
    entry_email.insert(0, usuario_selecionado[3])
    entry_data_nascimento.delete(0, tk.END)
    entry_data_nascimento.insert(0, usuario_selecionado[4])
    entry_pais.delete(0, tk.END)
    entry_pais.insert(0, usuario_selecionado[5])
    entry_estado.delete(0, tk.END)
    entry_estado.insert(0, usuario_selecionado[6])
    entry_cidade.delete(0, tk.END)
    entry_cidade.insert(0, usuario_selecionado[7])

    global usuario_id_selecionado
    usuario_id_selecionado = usuario_selecionado[0]

    tela_resultados.destroy()



def exibir_dados_usuario(usuario):
    tela_dados_usuario = tk.Toplevel(app)
    tela_dados_usuario.title("Dados do Usuário")

    tk.Label(tela_dados_usuario, text="Nome:").grid(row=0, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=f"{usuario[1]} {usuario[2]}").grid(row=0, column=1, padx=10, pady=5)

    tk.Label(tela_dados_usuario, text="Email:").grid(row=1, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=usuario[3]).grid(row=1, column=1, padx=10, pady=5)

    tk.Label(tela_dados_usuario, text="Data de Nascimento:").grid(row=2, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=usuario[4]).grid(row=2, column=1, padx=10, pady=5)

    tk.Label(tela_dados_usuario, text="País:").grid(row=3, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=usuario[5]).grid(row=3, column=1, padx=10, pady=5)

    tk.Label(tela_dados_usuario, text="Estado:").grid(row=4, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=usuario[6]).grid(row=4, column=1, padx=10, pady=5)

    tk.Label(tela_dados_usuario, text="Cidade:").grid(row=5, column=0, padx=10, pady=5)
    tk.Label(tela_dados_usuario, text=usuario[7]).grid(row=5, column=1, padx=10, pady=5)

    tk.Button(tela_dados_usuario, text="Fechar", command=tela_dados_usuario.destroy).grid(row=6, columnspan=2, pady=10)


def atualizar_usuario(usuario_id):
    primeiro_nome = entry_primeiro_nome.get()
    sobrenome = entry_sobrenome.get()
    email = entry_email.get()
    senha = entry_senha.get()
    data_nascimento = entry_data_nascimento.get()
    pais = entry_pais.get()
    estado = entry_estado.get()
    cidade = entry_cidade.get()

    try:
        cursor = conexao.cursor()

        cursor.execute('''
            UPDATE Usuario 
            SET primeiro_nome = %s, sobrenome = %s, email = %s, senha = %s, data_nascimento = %s, pais = %s, estado = %s, cidade = %s
            WHERE id = %s
        ''', (primeiro_nome, sobrenome, email, senha, data_nascimento, pais, estado, cidade, usuario_id))

        cursor.execute('DELETE FROM Telefone WHERE usuario_id = %s', (usuario_id,))
        for telefone in telefones:
            cursor.execute('''
                INSERT INTO Telefone (usuario_id, telefone)
                VALUES (%s, %s)
            ''', (usuario_id, telefone))

        conexao.commit()
        recarregar_dados()  
        cursor.execute("SELECT * FROM Usuario WHERE id = %s", (usuario_id,))
        usuario_atualizado = cursor.fetchone()
        print(usuario_atualizado)  # Verifique os dados aqui
        messagebox.showinfo("Sucesso", "Dados atualizados com sucesso!")

    except psycopg2.Error as e:
        messagebox.showerror("Erro", f"Erro ao atualizar dados: {e}")
        conexao.rollback()

def recarregar_dados():
    for row in tree.get_children():
        tree.delete(row)

    cursor.execute("SELECT * FROM Usuario")
    for usuario in cursor.fetchall():
        tree.insert("", "end", values=usuario)


def carregar_dados_usuario(nome_usuario):
    try:
        cursor = conexao.cursor()
        cursor.execute('SELECT id, primeiro_nome, sobrenome, email, data_nascimento, pais, estado, cidade FROM Usuario WHERE primeiro_nome = %s', (nome_usuario,))
        usuario = cursor.fetchone()

        if usuario:
            entry_primeiro_nome.delete(0, tk.END)
            entry_primeiro_nome.insert(0, usuario[1]) 
            entry_sobrenome.delete(0, tk.END)
            entry_sobrenome.insert(0, usuario[2])  
            entry_email.delete(0, tk.END)
            entry_email.insert(0, usuario[3])  
            entry_data_nascimento.delete(0, tk.END)
            entry_data_nascimento.insert(0, usuario[4]) 
            entry_pais.delete(0, tk.END)
            entry_pais.insert(0, usuario[5]) 
            entry_estado.delete(0, tk.END)
            entry_estado.insert(0, usuario[6])  
            entry_cidade.delete(0, tk.END)
            entry_cidade.insert(0, usuario[7]) 

            cursor.execute('SELECT telefone FROM Telefone WHERE usuario_id = %s', (usuario[0],))  # Usar o ID do usuário
            telefones.clear()
            listbox_telefones.delete(0, tk.END)
            for telefone in cursor.fetchall():
                telefones.append(telefone[0])
                listbox_telefones.insert(tk.END, telefone[0])
        if usuario:
            exibir_dados_usuario(usuario) 
        else:
            messagebox.showwarning("Aviso", "Usuário não encontrado.")
    except psycopg2.Error as e:
        messagebox.showerror("Erro", f"Erro ao carregar dados: {e}")




def limpar_campos():
    entry_primeiro_nome.delete(0, tk.END)
    entry_sobrenome.delete(0, tk.END)
    entry_email.delete(0, tk.END)
    entry_senha.delete(0, tk.END)
    entry_data_nascimento.delete(0, tk.END)
    entry_pais.delete(0, tk.END)
    entry_estado.delete(0, tk.END)
    entry_cidade.delete(0, tk.END)
    listbox_telefones.delete(0, tk.END)
    telefones.clear()

app = tk.Tk()
app.title("Cadastro de Usuário")

tk.Label(app, text="Primeiro Nome:").grid(row=0, column=0, padx=10, pady=5)
entry_primeiro_nome = tk.Entry(app)
entry_primeiro_nome.grid(row=0, column=1, padx=10, pady=5)

tk.Label(app, text="Sobrenome:").grid(row=1, column=0, padx=10, pady=5)
entry_sobrenome = tk.Entry(app)
entry_sobrenome.grid(row=1, column=1, padx=10, pady=5)

tk.Label(app, text="Email:").grid(row=2, column=0, padx=10, pady=5)
entry_email = tk.Entry(app)
entry_email.grid(row=2, column=1, padx=10, pady=5)

tk.Label(app, text="Senha:").grid(row=3, column=0, padx=10, pady=5)
entry_senha = tk.Entry(app, show="*")
entry_senha.grid(row=3, column=1, padx=10, pady=5)

tk.Label(app, text="Data de Nascimento (YYYY-MM-DD):").grid(row=4, column=0, padx=10, pady=5)
entry_data_nascimento = tk.Entry(app)
entry_data_nascimento.grid(row=4, column=1, padx=10, pady=5)

tk.Label(app, text="País:").grid(row=5, column=0, padx=10, pady=5)
entry_pais = tk.Entry(app)
entry_pais.grid(row=5, column=1, padx=10, pady=5)

tk.Label(app, text="Estado:").grid(row=6, column=0, padx=10, pady=5)
entry_estado = tk.Entry(app)
entry_estado.grid(row=6, column=1, padx=10, pady=5)

tk.Label(app, text="Cidade:").grid(row=7, column=0, padx=10, pady=5)
entry_cidade = tk.Entry(app)
entry_cidade.grid(row=7, column=1, padx=10, pady=5)

tk.Label(app, text="Telefone:").grid(row=8, column=0, padx=10, pady=5)
entry_telefone = tk.Entry(app)
entry_telefone.grid(row=8, column=1, padx=10, pady=5)

tk.Button(app, text="Adicionar Telefone", command=adicionar_telefone).grid(row=9, column=1, padx=10, pady=5)

tk.Label(app, text="Telefones:").grid(row=10, column=0, columnspan=2)
listbox_telefones = tk.Listbox(app, width=30, height=5)
listbox_telefones.grid(row=11, column=0, columnspan=2, padx=10, pady=5)

tk.Button(app, text="Cadastrar", command=cadastrar_usuario, bg="blue", fg="white").grid(row=12, column=0, columnspan=2, pady=10)
tk.Button(app, text="⚙️", command=abrir_tela_configuracoes, bg="gray", width=3, height=1).grid(row=13, column=2, padx=10, pady=10)

app.mainloop()